---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

Tensorflow Seq2Seq Tutorial:

https://www.tensorflow.org/tutorials/seq2seq/

In [1]:
%autosave 0

Autosave disabled


In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import itertools

In [3]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [4]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


In [5]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


In [6]:
vocabulary_size = len(string.ascii_lowercase) + 1 + 2 # [a-z] + ' ' + (symbol for 'GO') + (symbol for 'EOS')
first_letter = ord(string.ascii_lowercase[0])

GO = '@'

EOS = '#'

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  elif char == GO:
    return len(string.ascii_lowercase) + 1
  elif char == EOS:
    return len(string.ascii_lowercase) + 2
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if (dictid > 0) & (dictid <= len(string.ascii_lowercase)):
    return chr(dictid + first_letter - 1)
  elif dictid == len(string.ascii_lowercase) + 1:
    return GO
  elif dictid == len(string.ascii_lowercase) + 2:
    return EOS
  elif dictid == 0:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'), char2id(GO), char2id(EOS))
print(id2char(1), id2char(26), id2char(0), id2char(27), id2char(28))

Unexpected character: ï
1 26 0 0 27 28
a z   @ #


In [7]:
def str2id(string):
  """Convert a letter string to id list"""
  id = []
  for char in string:
    id.append(char2id(char))
  return id

In [8]:
def rev_str(string):
  """Convert a letter string to its mirror"""
  rev = ''
  for word in string.split(' '):
    rev = rev + word[::-1] + ' '
  return rev[0:-1]

print('Before reversal:')
print('the quick brown fox')
print('After reversal')
print(rev_str('the quick brown fox'))

Before reversal:
the quick brown fox
After reversal
eht kciuq nworb xof


In [9]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [10]:
def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

In [11]:
batch_size=64
num_unrollings=14 # The length of sequence to mirror in the training set

# In this problem, the length of input sequence and output sequence are exactly the same.
# However, in some other seq2seq problems, such as translation problems, the length of input and output sequence varies.

class Seq2SeqBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]

  def next(self):
    batches = []
    for i in range(self._batch_size):
      batch = ''
      for j in range(self._num_unrollings):
        batch += self._text[self._cursor[i]]
        self._cursor[i] = (self._cursor[i] + 1) % self._text_size
      batches.append(batch)
    return batches

In [12]:
train_batches = Seq2SeqBatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = Seq2SeqBatchGenerator(valid_text, 1, num_unrollings)
train_batches_example = train_batches.next()
valid_batches_example = valid_batches.next()
print(train_batches_example)
print(valid_batches_example)

['ons anarchists', 'when military ', 'lleria arches ', ' abbeys and mo', 'married urraca', 'hel and richar', 'y and liturgic', 'ay opened for ', 'tion from the ', 'migration took', 'new york other', 'he boeing seve', 'e listed with ', 'eber has proba', 'o be made to r', 'yer who receiv', 'ore significan', 'a fierce criti', ' two six eight', 'aristotle s un', 'ity can be los', ' and intracell', 'tion of the si', 'dy to pass him', 'f certain drug', 'at it will tak', 'e convince the', 'ent told him t', 'ampaign and ba', 'rver side stan', 'ious texts suc', 'o capitalize o', 'a duplicate of', 'gh ann es d hi', 'ine january ei', 'ross zero the ', 'cal theories c', 'ast instance t', ' dimensional a', 'most holy morm', 't s support or', 'u is still dis', 'e oscillating ', 'o eight subtyp', 'of italy langu', 's the tower co', 'klahoma press ', 'erprise linux ', 'ws becomes the', 'et in a nazi c', 'the fabian soc', 'etchy to relat', ' sharman netwo', 'ised emperor h', 'ting in politi', 'd neo la

In [13]:
def SeqBatch2Enc(batches):
    encoder_list = []
    for i in range(len(batches[0])):
        encoder = np.zeros(shape=(len(batches), vocabulary_size), dtype=np.float)
        for j in range(len(batches)):
            encoder[j, char2id(batches[j][i])] = 1.0
        encoder_list.append(encoder)
    return encoder_list

def SeqBatchMirror(batches):
    mirror = []
    for batch in batches:
        mirror.append(rev_str(batch))
    return mirror

In [14]:
def AppendGO(encoder_list):
    encoder_list_go = list()
    encoder_list_go = encoder_list_go + encoder_list
    """Append one-hot encoded Go term to the end of the encoder list"""
    go_encoder = np.zeros(shape=(len(encoder_list[0]), vocabulary_size), dtype=np.float)
    for i in xrange(len(go_encoder)):
        go_encoder[i, char2id(GO)] = 1.0
    encoder_list_go.append(go_encoder)
    return encoder_list_go

def AppendEOS(decoder_list):
    decoder_list_eos = list()
    decoder_list_eos = decoder_list_eos + decoder_list
    """Append one-hot encoded EOS term to the end of the decoder list"""
    eos_decoder = np.zeros(shape=(len(decoder_list[0]), vocabulary_size), dtype=np.float)
    for i in xrange(len(eos_decoder)):
        eos_decoder[i, char2id(EOS)] = 1.0
    decoder_list_eos.append(eos_decoder)
    return decoder_list_eos

In [15]:
print(SeqBatchMirror(train_batches_example))

['sno stsihcrana', 'nehw yratilim ', 'airell sehcra ', ' syebba dna om', 'deirram acarru', 'leh dna rahcir', 'y dna cigrutil', 'ya denepo rof ', 'noit morf eht ', 'noitargim koot', 'wen kroy rehto', 'eh gnieob eves', 'e detsil htiw ', 'rebe sah aborp', 'o eb edam ot r', 'rey ohw viecer', 'ero nacifingis', 'a ecreif itirc', ' owt xis thgie', 'eltotsira s nu', 'yti nac eb sol', ' dna llecartni', 'noit fo eht is', 'yd ot ssap mih', 'f niatrec gurd', 'ta ti lliw kat', 'e ecnivnoc eht', 'tne dlot mih t', 'ngiapma dna ab', 'revr edis nats', 'suoi stxet cus', 'o ezilatipac o', 'a etacilpud fo', 'hg nna se d ih', 'eni yraunaj ie', 'ssor orez eht ', 'lac seiroeht c', 'tsa ecnatsni t', ' lanoisnemid a', 'tsom yloh mrom', 't s troppus ro', 'u si llits sid', 'e gnitallicso ', 'o thgie pytbus', 'fo ylati ugnal', 's eht rewot oc', 'amohalk sserp ', 'esirpre xunil ', 'sw semoceb eht', 'te ni a izan c', 'eht naibaf cos', 'yhcte ot taler', ' namrahs owten', 'desi rorepme h', 'gnit ni itilop', 'd oen ni

In [16]:
num_nodes = 256

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    

  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  encoder_inputs = list()
  decoder_inputs = list()
  for i in xrange(num_unrollings + 1):
    encoder_inputs.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))   
    # Here, in this problem, the number of encoder inputs is the same to the number of decoder inputs
    # However, this might not be true for other problems, such as machine translation.
    decoder_inputs.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))    

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  # Encoder Section
  i = len(encoder_inputs) - 2
  while i >= 0:
    output, state = lstm_cell(encoder_inputs[i], output, state)
    i = i - 1
  output, state = lstm_cell(encoder_inputs[-1], output, state)
  outputs.append(output)
  # Decoder Section
  for i in decoder_inputs[0:-1]:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)
  
  # The 'outputs' includes EOS

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),saved_state.assign(state)]):
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, decoder_inputs)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)


  # Sampling and validation eval: 
  sample_inputs = list()
  for i in xrange(num_unrollings + 1):
    sample_inputs.append(tf.placeholder(tf.float32, shape=[1, vocabulary_size]))

  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))

  sample_outputs = list()
  sample_output = saved_sample_output
  sample_state = saved_sample_state 

  i = len(sample_inputs) - 2
  while i >= 0:
    sample_output, sample_state = lstm_cell(sample_inputs[i], sample_output, sample_state)
    i = i - 1
  sample_output, sample_state = lstm_cell(sample_inputs[-1], sample_output, sample_state)
  sample_outputs.append(sample_output)
    
  sample_output_logits = tf.nn.xw_plus_b(sample_output, w, b)
  sample_output_logits_softmax = tf.nn.softmax(sample_output_logits)

  for i in xrange(num_unrollings):
    sample_output, sample_state = lstm_cell(sample_output_logits_softmax, sample_output, sample_state)
    sample_outputs.append(sample_output)
    sample_output_logits = tf.nn.xw_plus_b(sample_output, w, b)
    sample_output_logits_softmax = tf.nn.softmax(sample_output_logits)

  sample_outputs_logits = tf.nn.xw_plus_b(tf.concat(0, sample_outputs), w, b)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(sample_outputs_logits)


In [17]:
num_steps = 20001
summary_frequency = 500

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    batches_mirror = SeqBatchMirror(batches)
    train_enc_inputs = SeqBatch2Enc(batches)
    train_dec_inputs = SeqBatch2Enc(batches_mirror)
    train_enc_inputs_go = AppendGO(train_enc_inputs)
    train_dec_inputs_eos = AppendEOS(train_dec_inputs)
    
    feed_dict = dict()
    # Add training data from batches to corresponding train_data position in the feed_dict
    for i in range(num_unrollings + 1):
      feed_dict[encoder_inputs[i]] = train_enc_inputs_go[i]
      feed_dict[decoder_inputs[i]] = train_dec_inputs_eos[i]
    # Train the model
    _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)

    mean_loss += l
    
    if step % summary_frequency == 0:
      print('=' * 60)
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(train_dec_inputs_eos))
      print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))

      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      num_validation = valid_size // num_unrollings
      for _ in range(num_validation):
        batches = valid_batches.next()
        batches_mirror = SeqBatchMirror(batches)
        valid_enc_inputs = SeqBatch2Enc(batches)
        valid_dec_inputs = SeqBatch2Enc(batches_mirror)
        valid_enc_inputs_go = AppendGO(valid_enc_inputs)
        valid_dec_inputs_eos = AppendEOS(valid_dec_inputs)
        
        feed_dict = dict()
        for i in range(num_unrollings + 1):
          feed_dict[sample_inputs[i]] = valid_enc_inputs_go[i]
        predictions = sample_prediction.eval(feed_dict = feed_dict)
        labels = np.concatenate(list(valid_dec_inputs_eos))
        valid_logprob = valid_logprob + logprob(predictions, labels)
      print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / num_validation)))    
    
      # Print sample out put
      sample = ['my name is lei']
      sample_enc_inputs = SeqBatch2Enc(sample)
      sample_enc_inputs_go = AppendGO(sample_enc_inputs)
      
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[sample_inputs[i]] = sample_enc_inputs_go[i]
      predictions = sample_prediction.eval(feed_dict = feed_dict)
      predicted_string = ''.join(element for element in characters(predictions))
        
      print('To mirror the string:')
      print(sample[0] + GO)
      print('After mirror by Seq2Seq LSTM:')
      print(predicted_string)

Initialized
Average loss at step 0: 3.392232 learning rate: 10.000000
Minibatch perplexity: 29.73
Validation set perplexity: 27.48
To mirror the string:
my name is lei@
After mirror by Seq2Seq LSTM:
#  #  #  #  #  
Average loss at step 500: 2.736578 learning rate: 10.000000
Minibatch perplexity: 11.50
Validation set perplexity: 22.84
To mirror the string:
my name is lei@
After mirror by Seq2Seq LSTM:
s sn e  #  #  #
Average loss at step 1000: 2.174900 learning rate: 10.000000
Minibatch perplexity: 6.37
Validation set perplexity: 14.98
To mirror the string:
my name is lei@
After mirror by Seq2Seq LSTM:
yl eno en  e  #
Average loss at step 1500: 1.570177 learning rate: 10.000000
Minibatch perplexity: 3.38
Validation set perplexity: 8.24
To mirror the string:
my name is lei@
After mirror by Seq2Seq LSTM:
ym emba  ni  ##
Average loss at step 2000: 1.103389 learning rate: 10.000000
Minibatch perplexity: 2.27
Validation set perplexity: 4.78
To mirror the string:
my name is lei@
After mirror 